# 2 Connecting to postgresql

A little introduction to the concepts of Postgresql and Pandas interaction.

To retrieve data from the postgresql database, I use [Pandas](http://pandas.pydata.org/pandas-docs/stable/index.html), which basically has Postgresql interaction built in and returns *dataframe* objects that can be easily displayed, filtered, plotted and so on. 

Interaction with the data happens via **queries** that are used to retrieve a subset of the data scattered throughout the database. These queries can have simple to complex forms and use their own (postgresql specific) language, which is quite easy to understand. The beauty about these queries is that they allow filtering on the flow, sorting, and - maybe most importantly - the merging of different tables matching certain criteria. Common tasks are for example the retrieval of data within a user specified range from a specific table or the data that has a common ID in two tables. Examples will be given below and in the next notebooks.

Authentication happens in two ways: Every computer has to have a password file saved which will be automatically accessed by postgres and will be used to log in to a specific database as a specific user. Additionally, the basic parameters are set in subfunctions (saved under `/database_helpers`).

This computer here has read-only access to the database, but your office computer has administrator privilege access because it needs also to change / edit data. 

### Basic setup

Every interaction starts with the basic import of libraries and functions ... 

In [2]:
# the following two lines indicate that external functions are auto-reloaded as soon as they change. 
# That's a nice trick!
%load_ext autoreload
%autoreload 2
# Print statements 
from __future__ import print_function # Python 2.x

In [3]:
# General stuff:
import sys
import argparse
import os
import json
import numpy as np
import math
import psycopg2
import cPickle
import numpy as np
import pandas as pd
from datetime import date
from tqdm import tqdm_notebook

# Plotting:
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
import matplotlib as mpl
%matplotlib inline

# External functions from subfolder /database_helpers. 
# as soon as you change something in there and press save, it will auto reload on next execution.
from database_helpers.psql_start import *
from database_helpers.create_tables import *
from database_helpers.write2tables import *

# register pickle type to retrieve binary data from database
psycopg2.extensions.register_type(psycopg2.extensions.new_type(psycopg2.BINARY.values, 'BINARY-PICKLE', cast_pickle))

Loaded analysis helpers: General


### Postgresql

Try a first handshake with the database via the function `test_connect()`

In [4]:
db_status = test_connect()
if db_status == False:
    print('Grrr... no database connection could be established.')
else:
    print('Yippiyeah! Database connection is established!')

Connecting to the PostgreSQL database...
Grrr... no database connection could be established.


### Query database with pandas

Every retrieval of data consists of **two steps**: Creation of sql query command, and execution of that command through pandas to retrieve a dataframe. To keep the data size small it is important to filter the data to be retrieved, but let's skip that for now (see below). Once loaded in a pandas dataframe, operations are run locally and additional filtering / manipulations can be applied.

In [5]:
# load the parameters to connect to data_1 
params = config()

In [6]:
%%time 
sql = "SELECT * FROM meta_tb" # sql command 
sql_db_pd = pd.read_sql_query(sql, psycopg2.connect(**params), index_col=None) # execute query and read into pandas dataframe

OperationalError: fe_sendauth: no password supplied


The code above loads the parameters (`params`) to connect to the database data_1. This only has to be done once per notebook. 
Then a sql command is created (`sql`) and it passed into a pandas `read_sql_query` command to retrieve a pandas dataframe object for that query. 

Let's look at that sql query:

`SELECT * FROM meta_tb` 
- SELECT - we want to read data, 
- we don't filter any columns (*) -- give me all the columns that are available 
- FROM meta_tb - which table to we want to read from

Let's see what we got. A convenient way to look at a pandas dataframe object is just to call it and pass a .head() or .tail() to it, which will show a snippet of the start or the end. An optional number within brackets defines how many rows you want to show. 

In [ ]:
sql_db_pd.head()

We have retrieved the complete meta_tb table and can inspect the output as dataframe object. Let's make both the sql query as well as the pandas function a bit more sophisticated:

### Simple filtering

In [ ]:
sql = "SELECT * FROM meta_tb WHERE session_ts > '1/6/2016' AND session_ts < '25/04/2017'"
sql_db_pd = pd.read_sql_query(sql, psycopg2.connect(**params), index_col=None,parse_dates=['session_ts','analysis_ts'])
sql_db_pd.sort_values(by='analysis_ts', ascending=False).head(3)

Executing that statement should have produced a view of the top 3 meta table entries sorted by analysis timestamp in descending order. 

What we integrated was a **filter** option (via `WHERE` in the sql query, filtering for a date range in the session timestamp column) and an additional parameter (`parse_dates`) in the pandas retrieval function, such that dates are parsed correctly. We also told pandas to sort the output for the analysis timestamp (`ascending=False`).

So in practice filtering can be done on the level of the database interaction within the sql command AND later on in the dataframe. The first filtering step is the most important one because we want to keep the dataset that is transferred via the local network small. But quering with a complex statement can put significant strain on the database server - so there is a trade off ... 

One more example:

In [ ]:
sql = "SELECT * FROM clusters_tb"
sql_db_pd = pd.read_sql_query(sql, psycopg2.connect(**params), index_col=None,parse_dates=['session_ts','analysis_ts'])
sql_db_pd.sort_values(by='analysis_ts', ascending=False).head(3)

Check the next notebook for data retrieval examples.